<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
See LICENSE file in the root directory for full license text.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# Notebook 2: Sensor Modalities in Autonomous Vehicles

**Author:** Milin Patel 
**Institution:** Hochschule Kempten

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/master/01_Perception_Systems/notebooks/02_sensor_technologies.ipynb)

---

## Learning Objectives

By the end of this notebook, you will:
- ✅ Understand different sensor modalities (Camera, LiDAR, Radar)
- ✅ Compare sensor capabilities and limitations
- ✅ Visualize 3D LiDAR point clouds
- ✅ Understand why sensor fusion is necessary
- ✅ Analyze sensor performance in different conditions

---

## Setup (Google Colab / Local)

Run this cell to install dependencies if running on Google Colab:

In [ ]:
import sys

# Check if running on Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Running on Google Colab - Installing dependencies...\n")
    !pip install -q opencv-python matplotlib numpy open3d plotly scipy
    
    # Clone repository for scripts
    !git clone -q https://github.com/milinpatel07/Autonomous-Driving_AI-Safety-and-Security.git
    sys.path.insert(0, '/content/Autonomous-Driving_AI-Safety-and-Security/AV_Perception_Safety_Workshop/Session_1_AI_Perception_Systems')
    print("✅ Setup complete!\n")
else:
    print("💻 Running locally\n")
    sys.path.insert(0, '..')

## 📚 Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Import workshop utilities
import sys
import os

# Add scripts directory to path
if IN_COLAB:
    scripts_path = '/content/Autonomous-Driving_AI-Safety-and-Security/AV_Perception_Safety_Workshop/Session_1_AI_Perception_Systems/scripts'
else:
    scripts_path = os.path.join(os.path.dirname(os.getcwd()), 'scripts') if 'notebooks' in os.getcwd() else 'scripts'

if os.path.exists(scripts_path):
    sys.path.insert(0, scripts_path)

try:
    from sensor_visualization import (
        visualize_pointcloud,
        visualize_sensor_comparison_table,
        load_sample_pointcloud,
        create_birds_eye_view
    )
    from dataset_loader import SyntheticDataGenerator
    print("✅ Workshop utilities loaded successfully!")
except ImportError as e:
    print(f"⚠️ Could not import utilities: {e}")
    print("⚠️ Falling back to inline implementations...")

    # Fallback: Define functions inline
    def visualize_sensor_comparison_table():
        """Display sensor comparison table"""
        data = [
            ['Sensor', 'Range', 'Resolution', 'Weather', 'Cost', 'Use Case'],
            ['Camera', '100m', 'High (2MP+)', '❌ Poor (rain/fog)', '💰 Low', 'Classification, signs'],
            ['LiDAR', '200m', 'High (0.1°)', '⚠️ Medium (fog)', '💰💰💰 High', '3D detection, mapping'],
            ['Radar', '250m', 'Low', '✅ Excellent', '💰💰 Medium', 'Speed, all-weather']
        ]

        fig, ax = plt.subplots(figsize=(14, 4))
        ax.axis('tight')
        ax.axis('off')

        table = ax.table(cellText=data, cellLoc='left', loc='center',
                        colWidths=[0.12, 0.12, 0.18, 0.22, 0.16, 0.20])
        table.auto_set_font_size(False)
        table.set_fontsize(11)
        table.scale(1, 2.5)

        for i in range(6):
            cell = table[(0, i)]
            cell.set_facecolor('#4CAF50')
            cell.set_text_props(weight='bold', color='white')

        for i in range(1, 4):
            for j in range(6):
                cell = table[(i, j)]
                cell.set_facecolor('#f0f0f0' if i % 2 == 0 else 'white')

        plt.title('📊 Autonomous Vehicle Sensor Comparison',
                 fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()

    def load_sample_pointcloud(sample_type='urban'):
        """Generate synthetic sample point cloud"""
        if sample_type == 'urban':
            ground = np.random.uniform([-30, 0, -1.8], [30, 50, -1.5], (5000, 3))
            buildings_left = np.random.uniform([-30, 5, -1.5], [-15, 40, 5], (3000, 3))
            buildings_right = np.random.uniform([15, 5, -1.5], [30, 40, 5], (3000, 3))
            cars = np.random.uniform([-10, 10, -1.5], [10, 30, 1], (1000, 3))
            points = np.vstack([ground, buildings_left, buildings_right, cars])
        else:
            points = np.random.uniform([-30, 0, -2], [30, 50, 5], (10000, 3))
        return points

    def create_birds_eye_view(points, image_size=(512, 512), x_range=(-40, 40),
                             y_range=(0, 80), z_range=(-3, 3)):
        """Create bird's eye view from point cloud"""
        h, w = image_size
        mask = (points[:, 2] >= z_range[0]) & (points[:, 2] <= z_range[1])
        points_filtered = points[mask]

        x_img = (points_filtered[:, 0] - x_range[0]) / (x_range[1] - x_range[0]) * w
        y_img = (points_filtered[:, 1] - y_range[0]) / (y_range[1] - y_range[0]) * h

        valid = (x_img >= 0) & (x_img < w) & (y_img >= 0) & (y_img < h)
        x_img = x_img[valid].astype(int)
        y_img = y_img[valid].astype(int)

        bev = np.zeros((h, w), dtype=np.float32)
        for x, y in zip(x_img, y_img):
            bev[h - 1 - y, x] += 1

        if bev.max() > 0:
            bev = np.clip(bev / bev.max(), 0, 1)
        return bev

    class SyntheticDataGenerator:
        """Simple synthetic data generator"""
        def generate_sample_image(self, scene_type='urban', size=(1242, 375)):
            h, w = size[1], size[0]
            image = np.zeros((h, w, 3), dtype=np.uint8)
            image[:h//3] = [135, 206, 235]  # Sky
            image[h//3:] = [105, 105, 105]  # Road
            for _ in range(np.random.randint(3, 8)):
                x, y = np.random.randint(50, w-50), np.random.randint(h//3, h-50)
                cv2.rectangle(image, (x, y), (x+80, y+100), (np.random.randint(0, 255),
                             np.random.randint(0, 255), np.random.randint(0, 255)), -1)
            return image

    print("✅ Fallback implementations loaded!")

# Set matplotlib style
plt.style.use('default')
%matplotlib inline

print("\n✅ All libraries imported!")

---

## Part 1: Sensor Comparison Table

Let's start by comparing the three main sensor modalities used in autonomous vehicles:

In [ ]:
# Display comprehensive sensor comparison
visualize_sensor_comparison_table()

### 📷 Camera Sensors

**Advantages:**
- High resolution (2MP to 8MP+)
- Rich semantic information (colors, textures, signs)
- Low cost (~$50-500)
- Excellent for object classification

**Disadvantages:**
- Poor performance in bad weather (rain, fog, darkness)
- No direct 3D information
- Affected by lighting conditions
- Limited depth perception

### LiDAR (Light Detection and Ranging)

**Advantages:**
- Accurate 3D measurements (cm-level precision)
- Long range (up to 200m)
- Works in darkness
- High angular resolution (0.1-0.2°)

**Disadvantages:**
- Very expensive ($1,000-$75,000)
- Affected by fog and heavy rain
- No color information
- Moving parts (mechanical scanners)

### Radar (Radio Detection and Ranging)

**Advantages:**
- All-weather operation (rain, fog, snow)
- Direct velocity measurement (Doppler)
- Long range (250m+)
- Moderate cost ($100-1,000)

**Disadvantages:**
- Low angular resolution
- Cannot classify objects well
- Ghost reflections
- No height information

---

## Part 2: 3D LiDAR Point Cloud Visualization

Let's visualize what LiDAR "sees" - a 3D point cloud!

In [ ]:
# Generate synthetic urban scene point cloud
print("📊 Generating synthetic urban scene...")
points = load_sample_pointcloud('urban')

print(f"✅ Generated {len(points):,} points")
print(f"   Point cloud shape: {points.shape}")
print(f"   X range: [{points[:, 0].min():.1f}, {points[:, 0].max():.1f}] meters")
print(f"   Y range: [{points[:, 1].min():.1f}, {points[:, 1].max():.1f}] meters")
print(f"   Z range: [{points[:, 2].min():.1f}, {points[:, 2].max():.1f}] meters")

In [ ]:
# Visualize point cloud (2D projection)
fig = plt.figure(figsize=(15, 5))

# Top view (X-Y)
ax1 = fig.add_subplot(131)
scatter = ax1.scatter(points[:, 0], points[:, 1], c=points[:, 2], 
                     cmap='viridis', s=0.5, alpha=0.6)
ax1.set_xlabel('X (meters)', fontsize=12)
ax1.set_ylabel('Y (meters)', fontsize=12)
ax1.set_title('Top View (Bird\'s Eye)', fontsize=14, fontweight='bold')
ax1.set_aspect('equal')
plt.colorbar(scatter, ax=ax1, label='Height (m)')

# Side view (Y-Z)
ax2 = fig.add_subplot(132)
ax2.scatter(points[:, 1], points[:, 2], c=points[:, 0], 
           cmap='plasma', s=0.5, alpha=0.6)
ax2.set_xlabel('Y (meters)', fontsize=12)
ax2.set_ylabel('Z (meters)', fontsize=12)
ax2.set_title('Side View', fontsize=14, fontweight='bold')
ax2.axhline(y=0, color='r', linestyle='--', label='Ground')
ax2.legend()

# Front view (X-Z)
ax3 = fig.add_subplot(133)
ax3.scatter(points[:, 0], points[:, 2], c=points[:, 1], 
           cmap='coolwarm', s=0.5, alpha=0.6)
ax3.set_xlabel('X (meters)', fontsize=12)
ax3.set_ylabel('Z (meters)', fontsize=12)
ax3.set_title('Front View', fontsize=14, fontweight='bold')
ax3.axhline(y=0, color='r', linestyle='--', label='Ground')
ax3.legend()

plt.tight_layout()
plt.show()

print("\n💡 The point cloud shows a 3D representation of the environment.")
print("   Each point represents a laser reflection from a surface.")

### Interactive 3D Visualization

Now let's create an interactive 3D view! (Note: Close the window to continue)

In [ ]:
# 3D matplotlib visualization
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Subsample for performance
subsample = points[::5]  # Every 5th point

# Color by height
colors = (subsample[:, 2] - subsample[:, 2].min()) / (subsample[:, 2].max() - subsample[:, 2].min())

scatter = ax.scatter(subsample[:, 0], subsample[:, 1], subsample[:, 2],
                    c=colors, cmap='viridis', s=1, alpha=0.6)

ax.set_xlabel('X (m)', fontsize=12)
ax.set_ylabel('Y (m)', fontsize=12)
ax.set_zlabel('Z (m)', fontsize=12)
ax.set_title('Interactive 3D Point Cloud\n(Rotate with mouse)', 
            fontsize=14, fontweight='bold')

# Set equal aspect ratio
max_range = np.array([
    subsample[:, 0].max()-subsample[:, 0].min(),
    subsample[:, 1].max()-subsample[:, 1].min(),
    subsample[:, 2].max()-subsample[:, 2].min()
]).max() / 2.0

mid_x = (subsample[:, 0].max()+subsample[:, 0].min()) * 0.5
mid_y = (subsample[:, 1].max()+subsample[:, 1].min()) * 0.5
mid_z = (subsample[:, 2].max()+subsample[:, 2].min()) * 0.5

ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

plt.colorbar(scatter, ax=ax, label='Height (normalized)', shrink=0.5)
plt.tight_layout()
plt.show()

---

## 🗺️ Part 3: Bird's Eye View (BEV)

Bird's eye view is commonly used for planning and navigation:

In [ ]:
# Create bird's eye view
bev = create_birds_eye_view(
    points,
    image_size=(512, 512),
    x_range=(-40, 40),
    y_range=(0, 80),
    z_range=(-2, 5)
)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# BEV grayscale
axes[0].imshow(bev, cmap='gray')
axes[0].set_title('Bird\'s Eye View (Grayscale)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('X (lateral)')
axes[0].set_ylabel('Y (forward)')
axes[0].grid(True, alpha=0.3)

# BEV with color
axes[1].imshow(bev, cmap='viridis')
axes[1].set_title('Bird\'s Eye View (Colored)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('X (lateral)')
axes[1].set_ylabel('Y (forward)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Bird's eye view is used for:")
print("   - Path planning")
print("   - Obstacle detection")
print("   - Parking maneuvers")
print("   - Lane keeping")

---

## 📷 Part 4: Camera vs LiDAR Comparison

Let's compare what camera and LiDAR "see" in the same scene:

In [ ]:
# Generate synthetic camera image
gen = SyntheticDataGenerator()
camera_image = gen.generate_sample_image('urban', size=(1242, 375))

# Visualize both modalities
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Camera image
axes[0, 0].imshow(camera_image)
axes[0, 0].set_title('📷 Camera View (RGB Image)', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')
axes[0, 0].text(10, 30, 'Rich semantic info\nColors, textures, signs', 
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
               fontsize=10, color='green', fontweight='bold')

# LiDAR top view
axes[0, 1].scatter(points[:, 0], points[:, 1], c=points[:, 2], 
                  cmap='viridis', s=0.3, alpha=0.6)
axes[0, 1].set_title('🌐 LiDAR Top View', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('X (m)')
axes[0, 1].set_ylabel('Y (m)')
axes[0, 1].set_aspect('equal')
axes[0, 1].text(-35, 45, 'Precise 3D geometry\nNo color info', 
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
               fontsize=10, color='blue', fontweight='bold')

# Bird's eye view
axes[1, 0].imshow(bev, cmap='viridis')
axes[1, 0].set_title('🗺️ Bird\'s Eye View (LiDAR)', fontsize=14, fontweight='bold')
axes[1, 0].axis('off')
axes[1, 0].text(20, 40, 'Top-down view\nUsed for planning', 
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
               fontsize=10, color='purple', fontweight='bold')

# 3D side view
axes[1, 1].scatter(points[:, 1], points[:, 2], c=points[:, 0], 
                  cmap='plasma', s=0.3, alpha=0.6)
axes[1, 1].set_title('🌐 LiDAR Side View', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Y (m)')
axes[1, 1].set_ylabel('Z (m)')
axes[1, 1].axhline(y=0, color='r', linestyle='--', linewidth=2, label='Ground')
axes[1, 1].legend()
axes[1, 1].text(5, 4, 'Height information\nGround detection', 
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
               fontsize=10, color='red', fontweight='bold')

plt.tight_layout()
plt.show()

---

## ☁️ Part 5: Weather Conditions Impact

Different sensors behave differently in various weather conditions:

In [ ]:
# Weather impact simulation
conditions = ['Clear', 'Rain', 'Fog', 'Night', 'Snow']
camera_performance = [95, 40, 30, 20, 35]
lidar_performance = [95, 75, 50, 95, 60]
radar_performance = [90, 95, 85, 90, 90]

x = np.arange(len(conditions))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width, camera_performance, width, label='📷 Camera', color='#FF6B6B')
bars2 = ax.bar(x, lidar_performance, width, label='🌐 LiDAR', color='#4ECDC4')
bars3 = ax.bar(x + width, radar_performance, width, label='📡 Radar', color='#45B7D1')

ax.set_xlabel('Weather Condition', fontsize=12, fontweight='bold')
ax.set_ylabel('Performance (%)', fontsize=12, fontweight='bold')
ax.set_title('Sensor Performance in Different Weather Conditions', 
            fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(conditions)
ax.legend(fontsize=12)
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{int(height)}%', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print("   ✅ Radar: Best all-weather performance")
print("   ⚠️ Camera: Struggles in rain, fog, and night")
print("   🌟 LiDAR: Good overall, but affected by fog")
print("\n💡 This is why sensor FUSION is critical for safe autonomous driving!")

---

## Part 6: Why Sensor Fusion?

By combining multiple sensors, we get the best of all worlds:

In [ ]:
# Sensor fusion benefits
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Individual sensor performance
scenarios = ['Object\nDetection', 'Distance\nEstimation', 'All-Weather\nOperation']
camera_scores = [90, 50, 40]
lidar_scores = [70, 95, 60]
radar_scores = [50, 85, 95]
fusion_scores = [95, 95, 90]

x = np.arange(len(scenarios))
width = 0.2

axes[0].bar(x - 1.5*width, camera_scores, width, label='Camera', color='#FF6B6B')
axes[0].bar(x - 0.5*width, lidar_scores, width, label='LiDAR', color='#4ECDC4')
axes[0].bar(x + 0.5*width, radar_scores, width, label='Radar', color='#45B7D1')
axes[0].bar(x + 1.5*width, fusion_scores, width, label='Fusion', color='#95E1D3')
axes[0].set_ylabel('Performance', fontsize=11, fontweight='bold')
axes[0].set_title('Performance Comparison', fontsize=12, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(scenarios)
axes[0].legend()
axes[0].set_ylim(0, 100)
axes[0].grid(True, alpha=0.3, axis='y')

# Redundancy
sensors_used = [1, 2, 3]
reliability = [85, 95, 99.5]
axes[1].plot(sensors_used, reliability, 'o-', linewidth=3, markersize=10, color='#F38181')
axes[1].fill_between(sensors_used, reliability, alpha=0.3, color='#F38181')
axes[1].set_xlabel('Number of Sensors', fontsize=11, fontweight='bold')
axes[1].set_ylabel('System Reliability (%)', fontsize=11, fontweight='bold')
axes[1].set_title('Redundancy Benefits', fontsize=12, fontweight='bold')
axes[1].set_xticks(sensors_used)
axes[1].set_ylim(80, 100)
axes[1].grid(True, alpha=0.3)
for i, (x, y) in enumerate(zip(sensors_used, reliability)):
    axes[1].text(x, y+0.5, f'{y}%', ha='center', fontweight='bold')

# Coverage improvement
categories = ['Camera Only', 'LiDAR Only', 'Radar Only', 'All Fused']
coverage = [65, 70, 60, 95]
colors_bar = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#95E1D3']
axes[2].barh(categories, coverage, color=colors_bar)
axes[2].set_xlabel('Environment Coverage (%)', fontsize=11, fontweight='bold')
axes[2].set_title('Detection Coverage', fontsize=12, fontweight='bold')
axes[2].set_xlim(0, 100)
axes[2].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(coverage):
    axes[2].text(v + 2, i, f'{v}%', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🎯 Sensor Fusion Benefits:")
print("   1. 🔄 Redundancy: If one sensor fails, others continue working")
print("   2. 📈 Better Performance: Combine strengths of each sensor")
print("   3. 🌐 Wider Coverage: Detect objects in more conditions")
print("   4. ✅ Higher Reliability: Reduce false positives/negatives")
print("   5. 🛡️ Safety: Critical for ISO 26262 compliance")

---

## Summary & Key Takeaways

### What We Learned:

1. **Three Main Sensors:**
 - 📷 Camera: Rich semantic info, weather-dependent
 - LiDAR: Accurate 3D, expensive, fog-sensitive
 - Radar: All-weather, low resolution

2. **Point Clouds:**
 - Represent 3D environment as collection of points
 - Each point has (x, y, z) coordinates + intensity
 - Can be visualized in different views (top, side, BEV)

3. **Sensor Fusion is Essential:**
 - No single sensor is perfect
 - Combining sensors improves reliability and coverage
 - Critical for safety-critical autonomous driving

### Next Steps:
- **Notebook 3:** Implement object detection with deep learning
- **Notebook 5:** Learn how to fuse sensor data

---

## ✅ Self-Check Questions

1. Which sensor works best in fog?
2. Why is LiDAR expensive?
3. What is a bird's eye view used for?
4. Name three benefits of sensor fusion.
5. Which sensor provides the best object classification?

<details>
<summary>Click for answers</summary>

1. Radar (radio waves penetrate fog)
2. Complex mechanical scanners, laser components, precision optics
3. Path planning, parking, lane keeping, obstacle detection
4. Redundancy, better performance, wider coverage
5. Camera (can see colors, textures, signs)
</details>

---

**🎉 Notebook Complete! Proceed to Notebook 3: Object Detection Demo**